<a href="https://colab.research.google.com/github/ollihansen90/Mathe-SH/blob/main/Labyrinthspiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import random

class Raum():
    def __init__(self, richtungen=[0.7,0.3,0.8,0.2]):
        self.oben = richtungen[0]
        self.links = richtungen[1]
        self.unten = richtungen[2]
        self.rechts = richtungen[3]
        self.bild = self.male_raum()

    def __str__(self):
        return self.bild

    def male_raum(self):
        s = 2*"\u2588"
        w = 2*" "

        if self.oben<0.5:
            ersteZeile = 5*s
        else:
            ersteZeile = 2*s+w+2*s
        zweiteZeile = vierteZeile = s+3*w+s

        dritteZeile = 3*w
        if self.links<0.5:
            dritteZeile = s+dritteZeile
        else:
            dritteZeile = w+dritteZeile
        
        if self.rechts<0.5:
            dritteZeile = dritteZeile+s
        else:
            dritteZeile = dritteZeile+w

        if self.unten<0.5:
            fuenfteZeile = 5*s
        else:
            fuenfteZeile = 2*s+w+2*s
        return ersteZeile+"\n"+zweiteZeile+"\n"+dritteZeile+"\n"+vierteZeile+"\n"+fuenfteZeile


class Karte():
    def __init__(self, hoehe, breite):
        self.raumliste = [[Raum([random.random() for i in range(4)]).bild for j in range(breite)] for k in range(hoehe)]
        print(self.raumliste)
    def male_karte(self):
        output = ""
        for zeile in self.raumliste:
            listeliste = [l.split("\n") for l in zeile]
            for i in range(len(listeliste[0])):
                for j in range(len(listeliste)):
                    output = output+listeliste[j][i]
                output = output+"\n"
        return output

r = [random.random() for i in range(4)]
raum1 = Raum(r)
#print(raum1)
k = Karte(hoehe=5, breite=4)

k.male_karte()

[['████  ████\n██      ██\n██      ██\n██      ██\n██████████', '████  ████\n██      ██\n        ██\n██      ██\n██████████', '██████████\n██      ██\n██      ██\n██      ██\n████  ████', '████  ████\n██      ██\n██      ██\n██      ██\n██████████'], ['████  ████\n██      ██\n██      ██\n██      ██\n██████████', '████  ████\n██      ██\n          \n██      ██\n██████████', '████  ████\n██      ██\n██      ██\n██      ██\n████  ████', '██████████\n██      ██\n        ██\n██      ██\n████  ████'], ['██████████\n██      ██\n          \n██      ██\n████  ████', '██████████\n██      ██\n          \n██      ██\n██████████', '████  ████\n██      ██\n        ██\n██      ██\n██████████', '████  ████\n██      ██\n██      ██\n██      ██\n██████████'], ['████  ████\n██      ██\n██      ██\n██      ██\n████  ████', '██████████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n██      ██\n██      ██\n████  ████', '██████████\n██      ██\n        ██\n██      ██\n████  ████'],

'████  ████████  ██████████████████  ████\n██      ████      ████      ████      ██\n██      ██        ████      ████      ██\n██      ████      ████      ████      ██\n████████████████████████  ██████████████\n████  ████████  ████████  ██████████████\n██      ████      ████      ████      ██\n██      ██          ██      ██        ██\n██      ████      ████      ████      ██\n████████████████████████  ████████  ████\n████████████████████████  ████████  ████\n██      ████      ████      ████      ██\n                            ████      ██\n██      ████      ████      ████      ██\n████  ██████████████████████████████████\n████  ██████████████████████████████████\n██      ████      ████      ████      ██\n██      ██        ████      ██        ██\n██      ████      ████      ████      ██\n████  ████████  ████████  ████████  ████\n██████████████  ████████  ██████████████\n██      ████      ████      ████      ██\n██      ████        ██      ██          \n██      ████      ████      ████ 

In [49]:
text1 = "Hallo\nMein Name ist Olli"
text2 = "Mir geht es heute gut.\nWie geht es dir?"

def verschmelze(str1, str2):
    output = ""
    liste1 = str1.split("\n")
    liste2 = str2.split("\n")
    for i in range(len(liste1)):
        output = output+liste1[i]+liste2[i]+"\n"
    return output

def verschmelze2(liste):
    output = ""
    listeliste = [l.split("\n") for l in liste]
    for i in range(len(listeliste[0])):
        for j in range(len(listeliste)):
            output = output+listeliste[j][i]
        output = output+"\n"
    return output

def verschmelze3(zeilenliste):
    output = ""
    for zeile in zeilenliste:
        listeliste = [l.split("\n") for l in zeile]
        for i in range(len(listeliste[0])):
            for j in range(len(listeliste)):
                output = output+listeliste[j][i]
            output = output+"\n"
    return output

#print(verschmelze(text1, text2))

r = [random.random() for i in range(4)]
raum1 = Raum(r)
r = [random.random() for i in range(4)]
raum2 = Raum(r)
raumliste = [Raum([random.random() for i in range(4)]).bild for j in range(8)]
karte = [[Raum([random.random() for i in range(4)]).bild for j in range(8)] for k in range(8)]
#print(raumliste)
#print(verschmelze(raum1.bild, raum2.bild))
#print(verschmelze2(raumliste))
print(karte)
print(verschmelze3(karte))

[['████  ████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n        ██\n██      ██\n██████████', '██████████\n██      ██\n██        \n██      ██\n██████████', '██████████\n██      ██\n██      ██\n██      ██\n██████████', '██████████\n██      ██\n██        \n██      ██\n██████████', '████  ████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n          \n██      ██\n████  ████'], ['██████████\n██      ██\n          \n██      ██\n██████████', '██████████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n        ██\n██      ██\n████  ████', '██████████\n██      ██\n          \n██      ██\n████  ████', '██████████\n██      ██\n██      ██\n██      ██\n██████████', '████  ████\n██      ██\n          \n██      ██\n████  ████', '██████████\n██      ██\n██      ██\n██      ██\n████  ████', '██████████\n██      ██\n          \n██      ██\n████  ████'], ['█